In [1]:
%run ./process_data_new.py

:::::::::::::::::::: PREPARING ::::::::::::::::::::
[INFO]: Featurizing 45 files: 
  Featurizing 1.mp3 -> .........................DONE
  Featurizing 10.mp3 -> .........................DONE
  Featurizing 11.mp3 -> .........................DONE
  Featurizing 12.mp3 -> .........................DONE
  Featurizing 13.mp3 -> .........................DONE
  Featurizing 14.mp3 -> .........................DONE
  Featurizing 15.mp3 -> .........................DONE
  Featurizing 16.mp3 -> .........................DONE
  Featurizing 17.mp3 -> .........................DONE
  Featurizing 18.mp3 -> .........................DONE
  Featurizing 19.mp3 -> .........................DONE
  Featurizing 2.mp3 -> .........................DONE
  Featurizing 20.mp3 -> .........................DONE
  Featurizing 21.mp3 -> .........................DONE
  Featurizing 22.mp3 -> .........................DONE
  Featurizing 23.mp3 -> .........................DONE
  Featurizing 24.mp3 -> .........................DONE
 

In [23]:
import tensorflow as tf
import numpy as np
import pandas as pd
import sklearn as sk
import joblib
import random
import gc
import matplotlib.pyplot as plt

In [24]:
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available: 0


In [25]:
df = pd.read_hdf("./dataset.h5", key="df")
df

,Cens_C,Cens_Db,Cens_D,Cens_Eb,Cens_E,Cens_F,Cens_Gb,Cens_G,Cens_Ab,Cens_A,Cens_Bb,Cens_B,chord,filename
0,0.458163,0.228498,0.335618,0.385245,0.280831,0.562060,0.101404,0.142517,0.211683,0.000000,0.081103,0.007991,FMin7,../Datasets/1.mp3
1,0.456882,0.228481,0.329903,0.386718,0.270461,0.563033,0.104248,0.142325,0.223759,0.000000,0.096144,0.008832,FMin7,../Datasets/1.mp3
2,0.454308,0.229081,0.324611,0.388377,0.260669,0.563478,0.107217,0.142526,0.235052,0.000000,0.110957,0.009594,FMin7,../Datasets/1.mp3
3,0.450574,0.230340,0.319815,0.390179,0.251621,0.563336,0.110333,0.143008,0.245606,0.000000,0.125269,0.010269,FMin7,../Datasets/1.mp3
4,0.445894,0.232287,0.315578,0.392097,0.243367,0.562581,0.113524,0.143594,0.255459,0.000000,0.138864,0.010856,FMin7,../Datasets/1.mp3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,0.207725,0.076254,0.397630,0.012440,0.081124,0.025550,0.113143,0.602074,0.253705,0.415162,0.012520,0.416366,AMin7,../Datasets/9.mp3
109,0.200846,0.072857,0.392787,0.013882,0.080756,0.028333,0.115661,0.609691,0.257939,0.416193,0.013882,0.409283,AMin7,../Datasets/9.mp3
110,0.194574,0.068975,0.387773,0.015407,0.080133,0.031256,0.117944,0.616832,0.261322,0.417602,0.015407,0.402578,AMin7,../Datasets/9.mp3
111,0.188929,0.064633,0.382609,0.017022,0.079286,0.034334,0.119918,0.623479,0.263771,0.419261,0.017022,0.396526,AMin7,../Datasets/9.mp3


In [26]:
unique_file = df["filename"].unique()
train_file, test_file = sk.model_selection.train_test_split(
    unique_file,
    random_state=42,
    test_size=0.2,
)
test_file, val_file = sk.model_selection.train_test_split(
    test_file,
    random_state=42,
    test_size=0.5,
)

df_train = df[df["filename"].isin(train_file)]
df_val = df[df["filename"].isin(val_file)]
df_test = df[df["filename"].isin(test_file)]

In [27]:
print("Chords")
print("\n## TRAIN ##", df_train["chord"].value_counts())
print("\n## VAL ##", df_val["chord"].value_counts())
print("\n## TEST ##", df_test["chord"].value_counts())

Chords

## TRAIN ## chord
AbMin7    15671
CMin7     15642
FMin7     15555
GbMin7    15474
DbMin7    15298
EMin7     14917
AMin7     14859
BbMin7    14805
GMin7     14679
EbMin7    14666
DMin7     14658
BMin7     14601
AMaj7     14174
AbMaj7    13936
EbMaj7    13895
EMaj7     13889
BbMaj7    13815
GbMaj7    13661
BMaj7     13602
GMaj7     13564
DbMaj7    13555
DMaj7     13547
CMaj7     13523
FMaj7     13339
G7         3454
C7         3395
Bb7        3386
B7         3360
Db7        3232
Ab7        3230
D7         3200
Eb7        3185
E7         3175
A7         3162
Gb7        3155
F7         3141
Name: count, dtype: int64

## VAL ## chord
DMaj7     4538
GMaj7     3987
FMaj7     3514
DbMaj7    3511
EbMaj7    3497
BbMaj7    3489
GbMaj7    3489
BMaj7     3489
CMaj7     3489
AMaj7     3474
EMaj7     3474
AbMaj7    3474
EMin7     2223
AMin7     1804
DMin7     1774
BbMin7    1758
GMin7     1756
CMin7     1753
GbMin7    1746
AbMin7    1741
EbMin7    1732
FMin7     1731
DbMin7    1716
BMin7     

In [28]:
# y = df["chord"]
# X = df.drop(columns="chord")
# encoder = sk.preprocessing.LabelEncoder()
# y_encoded = encoder.fit_transform(y)

# joblib.dump(encoder, "./encoder.xz")

def get_Xy(df: pd.DataFrame):
    return df.drop(["chord", "filename"], axis=1), df["chord"]

X_train, y_train = get_Xy(df_train)
X_val, y_val = get_Xy(df_val)
X_test, y_test = get_Xy(df_test)

encoder = sk.preprocessing.LabelEncoder()
y_train_encoded = encoder.fit_transform(y_train)
y_val_encoded = encoder.transform(y_val)
y_test_encoded = encoder.transform(y_test)

joblib.dump(encoder, "./encoder.xz")

['./encoder.xz']

In [29]:
# del df
# gc.collect()

# LSTM

In [30]:
# SEQUENCE_LEN = 20 # 0.1 * 20.0 = 2 sec of sequence data

# def make_seq(X, y_encoded):
#     X_seq, y_encoded_seq = None, None
#     X_seq_list = []
#     y_encoded_seq_list = []
#     for i in range(len(X) - SEQUENCE_LEN + 1):
#         X_seq_list.append(X.values[i : i + SEQUENCE_LEN, :])
#         y_encoded_seq_list.append(y_encoded[i + SEQUENCE_LEN - 1])
    
#     return np.array(X_seq_list), np.array(y_encoded_seq_list)

# X_train, y_train_encoded = make_seq(X_train, y_train_encoded)
# X_val, y_val_encoded = make_seq(X_val, y_val_encoded)
# X_test, y_test_encoded = make_seq(X_test, y_test_encoded)

# print("TRAIN")
# print("X sequence shape: ", X_train.shape)
# print("y sequence shape: ", y_train_encoded.shape)

# print("\nVAL")
# print("X sequence shape: ", X_val.shape)
# print("y sequence shape: ", y_val_encoded.shape)

# print("\nTEST")
# print("X sequence shape: ", X_test.shape)
# print("y sequence shape: ", y_test_encoded.shape)

# gc.collect()

In [31]:
# X_seq_train, X_seq_test, y_seq_train, y_seq_test = sk.model_selection.train_test_split(
#     X_seq,
#     y_encoded_seq,
#     test_size=0.2,
#     random_state=42,
#     stratify=y_encoded_seq,
# )

# print(f"X_train: {len(X_seq_train)}")
# print(f"y_train: {len(y_seq_train)}")
# print(f"X_test:  {len(X_seq_test)}")
# print(f"y_test:  {len(y_seq_test)}")

# del X_seq
# del y_encoded_seq
# gc.collect()

In [32]:
# print("Total feature: ", X_train.shape[2])
# print("Total class:   ", len(encoder.classes_))

In [33]:
class_weights = sk.utils.class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train,
)
class_weight_dict = dict(enumerate(class_weights))

In [34]:
layers = [
    (16, 16),
    (32, 32),
    (64, 64)
]

seq_lens = [5, 10, 20]
    
early_stopping = tf.keras.callbacks.EarlyStopping(
    patience=10,
    restore_best_weights=True,
    # monitor="val_accuracy"
)

def make_seq(X, y_encoded, seq_len):
    X_seq, y_encoded_seq = None, None
    X_seq_list = []
    y_encoded_seq_list = []
    for i in range(len(X) - seq_len + 1):
        X_seq_list.append(X.values[i : i + seq_len, :])
        y_encoded_seq_list.append(y_encoded[i + seq_len - 1])
    
    return np.array(X_seq_list), np.array(y_encoded_seq_list)

# models = []
for seq_len in seq_lens:
    X_train_seq, y_train_encoded_seq = make_seq(X_train, y_train_encoded, seq_len)
    X_val_seq, y_val_encoded_seq = make_seq(X_val, y_val_encoded, seq_len)
    X_test_seq, y_test_encoded_seq = make_seq(X_test, y_test_encoded, seq_len)
    
    for layer_1, layer_2 in layers:
        model_lstm = tf.keras.Sequential([
            # tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True, input_shape=(SEQUENCE_LEN, X_seq_train.shape[2]))),
            tf.keras.layers.Bidirectional(
                tf.keras.layers.LSTM(
                    layer_1,
                    return_sequences=True,
                    input_shape=(seq_len, X_train_seq.shape[2]),
                    kernel_regularizer=tf.keras.regularizers.l2(1e-3),
                    recurrent_regularizer=tf.keras.regularizers.l2(1e-3),
                    recurrent_dropout=0.25,
                )
            ),
            tf.keras.layers.Dropout(0.25),
                
            tf.keras.layers.Bidirectional(
                tf.keras.layers.LSTM(
                    layer_2,
                    return_sequences=False,
                    kernel_regularizer=tf.keras.regularizers.l2(1e-3),
                    recurrent_regularizer=tf.keras.regularizers.l2(1e-3),
                    recurrent_dropout=0.25,
                )
            ),
            tf.keras.layers.Dropout(0.25),
        
            tf.keras.layers.Dense(
                len(encoder.classes_),
                activation='softmax',
                # kernel_regularizer=tf.keras.regularizers.l2(1e-2)
            ),
        ])
        
        model_lstm.compile(
            optimizer="adam",
            loss="sparse_categorical_crossentropy",
            metrics=["accuracy"],
        )
        
        # model_lstm.summary()
        
        history = model_lstm.fit(
            # X_seq_train,
            # y_seq_train,
            X_train_seq,
            y_train_encoded_seq,
            epochs=500,
            batch_size=32,
            # validation_split=0.1,
            validation_data=(X_val_seq, y_val_encoded_seq),
            verbose=1,
            callbacks=[early_stopping],
            class_weight=class_weight_dict,
        )
    
        # models.append(model_lstm)
        
        model_lstm.save(f"./Models/model_lstm_{layer_1}_{layer_2}_seq_{seq_len}.keras")
        pd.DataFrame(history.history).to_csv(f"./History/model_lstm_{layer_1}_{layer_2}_seq_{seq_len}_history.csv", index=False)

        print(f"::::: model_lstm_{layer_1}_{layer_2}_seq_{seq_len} :::::")
        print(model_lstm.evaluate(X_test_seq, y_test_encoded_seq))
        y_pred = np.argmax(model_lstm.predict(X_test_seq), axis=1)
        print(
            sk.metrics.classification_report(y_test_encoded_seq, y_pred, target_names=encoder.classes_)
        )
        print("")
    
        gc.collect()

/usr/lib/python3.13/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/500
12013/12013 ━━━━━━━━━━━━━━━━━━━━ 65s 5ms/step - accuracy: 0.4031 - loss: 2.2809 - val_accuracy: 0.6739 - val_loss: 1.2393
Epoch 2/500
12013/12013 ━━━━━━━━━━━━━━━━━━━━ 61s 5ms/step - accuracy: 0.5109 - loss: 1.7479 - val_accuracy: 0.6751 - val_loss: 1.2096
Epoch 3/500
12013/12013 ━━━━━━━━━━━━━━━━━━━━ 62s 5ms/step - accuracy: 0.5185 - loss: 1.7125 - val_accuracy: 0.6625 - val_loss: 1.2186
Epoch 4/500
12013/12013 ━━━━━━━━━━━━━━━━━━━━ 62s 5ms/step - accuracy: 0.5194 - loss: 1.7088 - val_accuracy: 0.6580 - val_loss: 1.2083
Epoch 5/500
12013/12013 ━━━━━━━━━━━━━━━━━━━━ 64s 5ms/step - accuracy: 0.5259 - loss: 1.6773 - val_accuracy: 0.6581 - val_loss: 1.2255
Epoch 6/500
12013/12013 ━━━━━━━━━━━━━━━━━━━━ 63s 5ms/step - accuracy: 0.5278 - loss: 1.6723 - val_accuracy: 0.6536 - val_loss: 1.2323
Epoch 7/500
12013/12013 ━━━━━━━━━━━━━━━━━━━━ 63s 5ms/step - accuracy: 0.5295 - loss: 1.6664 - val_accuracy: 0.6590 - val_loss: 1.2154
Epoch 8/500
12013/12013 ━━━━━━━━━━━━━━━━━━━━ 62s 5ms/step - ac

/usr/lib/python3.13/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


12013/12013 ━━━━━━━━━━━━━━━━━━━━ 76s 6ms/step - accuracy: 0.4642 - loss: 2.0770 - val_accuracy: 0.6703 - val_loss: 1.2368
Epoch 2/500
12013/12013 ━━━━━━━━━━━━━━━━━━━━ 71s 6ms/step - accuracy: 0.5444 - loss: 1.6371 - val_accuracy: 0.6648 - val_loss: 1.2156
Epoch 3/500
12013/12013 ━━━━━━━━━━━━━━━━━━━━ 71s 6ms/step - accuracy: 0.5522 - loss: 1.5967 - val_accuracy: 0.6893 - val_loss: 1.1834
Epoch 4/500
12013/12013 ━━━━━━━━━━━━━━━━━━━━ 71s 6ms/step - accuracy: 0.5583 - loss: 1.5713 - val_accuracy: 0.6651 - val_loss: 1.2305
Epoch 5/500
12013/12013 ━━━━━━━━━━━━━━━━━━━━ 71s 6ms/step - accuracy: 0.5608 - loss: 1.5480 - val_accuracy: 0.6694 - val_loss: 1.2036
Epoch 6/500
12013/12013 ━━━━━━━━━━━━━━━━━━━━ 72s 6ms/step - accuracy: 0.5647 - loss: 1.5332 - val_accuracy: 0.6659 - val_loss: 1.2212
Epoch 7/500
12013/12013 ━━━━━━━━━━━━━━━━━━━━ 72s 6ms/step - accuracy: 0.5661 - loss: 1.5323 - val_accuracy: 0.6777 - val_loss: 1.2052
Epoch 8/500
12013/12013 ━━━━━━━━━━━━━━━━━━━━ 72s 6ms/step - accuracy: 0.56

/usr/lib/python3.13/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


...

Epoch 1/500
12012/12012 ━━━━━━━━━━━━━━━━━━━━ 189s 15ms/step - accuracy: 0.3703 - loss: 2.4017 - val_accuracy: 0.6049 - val_loss: 1.3760
Epoch 2/500
12012/12012 ━━━━━━━━━━━━━━━━━━━━ 185s 15ms/step - accuracy: 0.4939 - loss: 1.8172 - val_accuracy: 0.6334 - val_loss: 1.2850
Epoch 3/500
12012/12012 ━━━━━━━━━━━━━━━━━━━━ 186s 15ms/step - accuracy: 0.5088 - loss: 1.7435 - val_accuracy: 0.6424 - val_loss: 1.2586
Epoch 4/500
12012/12012 ━━━━━━━━━━━━━━━━━━━━ 186s 15ms/step - accuracy: 0.5190 - loss: 1.7033 - val_accuracy: 0.6340 - val_loss: 1.2699
Epoch 5/500
12012/12012 ━━━━━━━━━━━━━━━━━━━━ 186s 16ms/step - accuracy: 0.5230 - loss: 1.6858 - val_accuracy: 0.6613 - val_loss: 1.2283
Epoch 6/500
12012/12012 ━━━━━━━━━━━━━━━━━━━━ 187s 16ms/step - accuracy: 0.5267 - loss: 1.6658 - val_accuracy: 0.6397 - val_loss: 1.2622
Epoch 7/500
12012/12012 ━━━━━━━━━━━━━━━━━━━━ 187s 16ms/step - accuracy: 0.5299 - loss: 1.6603 - val_accuracy: 0.6603 - val_loss: 1.2320
Epoch 8/500
12012/12012 ━━━━━━━━━━━━━━━━━━━━ 188

/usr/lib/python3.13/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/500
12012/12012 ━━━━━━━━━━━━━━━━━━━━ 211s 17ms/step - accuracy: 0.4175 - loss: 2.2461 - val_accuracy: 0.6425 - val_loss: 1.2881
Epoch 2/500
12012/12012 ━━━━━━━━━━━━━━━━━━━━ 206s 17ms/step - accuracy: 0.5346 - loss: 1.6837 - val_accuracy: 0.6564 - val_loss: 1.2635
Epoch 3/500
12012/12012 ━━━━━━━━━━━━━━━━━━━━ 208s 17ms/step - accuracy: 0.5504 - loss: 1.6156 - val_accuracy: 0.6592 - val_loss: 1.2617
Epoch 4/500
12012/12012 ━━━━━━━━━━━━━━━━━━━━ 209s 17ms/step - accuracy: 0.5586 - loss: 1.5738 - val_accuracy: 0.6737 - val_loss: 1.2236
Epoch 5/500
12012/12012 ━━━━━━━━━━━━━━━━━━━━ 210s 17ms/step - accuracy: 0.5625 - loss: 1.5617 - val_accuracy: 0.6753 - val_loss: 1.2199
Epoch 6/500
12012/12012 ━━━━━━━━━━━━━━━━━━━━ 210s 18ms/step - accuracy: 0.5659 - loss: 1.5498 - val_accuracy: 0.6575 - val_loss: 1.2592
Epoch 7/500
12012/12012 ━━━━━━━━━━━━━━━━━━━━ 212s 18ms/step - accuracy: 0.5705 - loss: 1.5256 - val_accuracy: 0.6590 - val_loss: 1.2575
Epoch 8/500
12012/12012 ━━━━━━━━━━━━━━━━━━━━ 213

/usr/lib/python3.13/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/500
12012/12012 ━━━━━━━━━━━━━━━━━━━━ 250s 20ms/step - accuracy: 0.4540 - loss: 2.1363 - val_accuracy: 0.6265 - val_loss: 1.4038
Epoch 2/500
12012/12012 ━━━━━━━━━━━━━━━━━━━━ 247s 21ms/step - accuracy: 0.5551 - loss: 1.6629 - val_accuracy: 0.6533 - val_loss: 1.3311
Epoch 3/500
12012/12012 ━━━━━━━━━━━━━━━━━━━━ 249s 21ms/step - accuracy: 0.5707 - loss: 1.5946 - val_accuracy: 0.6441 - val_loss: 1.3732
Epoch 4/500
12012/12012 ━━━━━━━━━━━━━━━━━━━━ 250s 21ms/step - accuracy: 0.5774 - loss: 1.5637 - val_accuracy: 0.6646 - val_loss: 1.3265
Epoch 5/500
12012/12012 ━━━━━━━━━━━━━━━━━━━━ 252s 21ms/step - accuracy: 0.5808 - loss: 1.5482 - val_accuracy: 0.6443 - val_loss: 1.3663
Epoch 6/500
12012/12012 ━━━━━━━━━━━━━━━━━━━━ 252s 21ms/step - accuracy: 0.5858 - loss: 1.5286 - val_accuracy: 0.6656 - val_loss: 1.3168
Epoch 7/500
12012/12012 ━━━━━━━━━━━━━━━━━━━━ 254s 21ms/step - accuracy: 0.5889 - loss: 1.5184 - val_accuracy: 0.6476 - val_loss: 1.3573
Epoch 8/500
12012/12012 ━━━━━━━━━━━━━━━━━━━━ 255